<h1>**Modelos Ingeniería Logística**</h1>

**1.- 3 Modelos del TSP utilizando las variables u, j o f para evitar los subciclos**

1.1.- Primer modelo (Usando la variable *u* para conocer la posición que ocupa cada nodo en la ruta final)

In [0]:
!pip install pulp
from pulp import *
!pip install ortools
from ortools.linear_solver import pywraplp
import random

n = 50
todos = range(n)
otros = range(1,n)
random.seed(12345)
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
EPS = 0.001

def TSP1():
    solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
    u = {}
    for i in otros:
        u[i] = solver.IntVar(0.0, solver.infinity(), 'u[%i]' % i)  

    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  u[j] >= u[i] + x[i,j] - (n-2)*(1-x[i,j]) + (n-3)*x[j,i]   ) for i in otros for j in otros if j!=i ]
        
    solver.Solve()

    print('Distancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('Ruta de mínima distancia:')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('De %d a %d con distancia=%d Km'  %(i,j,dist[i,j]))

TSP1()


1.2.- Segundo modelo (Usando la variable *v* para llevar el control de que es nodo es visitado antes que otro)

In [0]:
!pip install pulp
from pulp import *
!pip install ortools
from ortools.linear_solver import pywraplp
import random

n = 50
todos = range(n)
otros = range(1,n)
random.seed(12345)
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
EPS = 0.001

def TSP2():
    solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))              
    v = {}
    for i in todos:
        for j in todos:
            if i!=j:
                v[i,j] = solver.BoolVar('v[%i,%i]' % (i,j))

    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    
    [ solver.Add(  v[i,j] + v[j,i] == 1  )  for i in otros for j in otros if i!=j ]
    [ solver.Add(  x[i,k] + v[i,j] + v[j,k] <= v[i,k] + 1  )  for i in otros for j in otros for k in todos if i!=j and j!=k and k!=i ]
    [ solver.Add(  v[i,j] == 0 or v[i,j] == 1)  for i in todos for j in todos if i!=j ]
    [ solver.Add(  x[i,j] <= v[i,j]  )  for i in otros for j in todos if i!=j ]
      
    solver.Solve()

    print('Distancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('Ruta de mínima distancia:')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('De %d a %d con distancia=%d Km'  %(i,j,dist[i,j]))

TSP2()

1.3.- Tercer modelo (Usando la variable *f* para llevar el control de la cantidad de euros que llevo por un arco)

In [0]:
!pip install pulp
from pulp import *
!pip install ortools
from ortools.linear_solver import pywraplp
import random

n = 50
todos = range(n)
otros = range(1,n)
random.seed(12345)
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
EPS = 0.001

def TSP3():
    solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
    f = {}
    for i in todos:
        for j in todos:
            if i!=j:
                f[i,j] = solver.IntVar(0.0, solver.infinity(), 'f[%i,%i]' % (i,j))  

    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    
    [ solver.Add(  solver.Sum(f[i,j] for i in todos if i!=j) - solver.Sum(f[j,k] for k in todos if j!=k) == 1  )  for j in otros]
    [ solver.Add(  0 <= f[i,j]  )  for i in todos for j in todos if i!=j]
    [ solver.Add(  f[i,j] <= (n - 1) * x[i,j]) for i in todos for j in todos if i!=j]
        
    solver.Solve()

    print('Distancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('Ruta de mínima distancia:')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('De %d a %d con distancia=%d Km'  %(i,j,dist[i,j]))

TSP3()

**2.- Modelo del TSP con Relaciones de Precedencia**

In [0]:
!pip install pulp
from pulp import *
!pip install ortools
from ortools.linear_solver import pywraplp
import random

n = 50
todos = range(n)
otros = range(1,n)
random.seed(12345)
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
EPS = 0.001

def TSP4():
    solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
    v = {}
    for i in todos:
        for j in todos:
            if i!=j:
                v[i,j] = solver.BoolVar('v[%i,%i]' % (i,j))  

    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    
    [ solver.Add(  v[i,j] + v[j,i] == 1  )  for i in todos for j in todos if i!=j ]
    [ solver.Add(  v[i,j] + v[j,k] <= v[i,k] + 1  )  for i in todos for j in todos for k in todos if i!=j and j!=k and k!=i ]
    [ solver.Add(  x[i,j] <= v[i,j]  )  for i in todos for j in todos if i!=j ]
    [ solver.Add(  solver.Sum(v[i,j] for j in todos if i!=j) == 1  for i in todos if i!=j)]
    
    solver.Solve()

    print('Distancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('Ruta de mínima distancia:')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('De %d a %d con distancia=%d Km'  %(i,j,dist[i,j]))

TSP4()

**3.- Modelo del TSP con recogidas y entregas de un mercancia**

In [0]:
!pip install pulp
from pulp import *
!pip install ortools
from ortools.linear_solver import pywraplp
import random

n = 50
todos = range(n)
otros = range(1,n)
random.seed(12345)
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
demanda= { (i) : random.randint(1,10) for i in otros }
Q = 500 #Cantidad máxima de mercancia en el camión transportista
EPS = 0.001

def TSP5():
    solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
    u = {}
    for i in otros:
        u[i] = solver.IntVar(0.0, solver.infinity(), 'u[%i]' % i) 
    f = {}
    for i in todos:
        for j in todos:
            if i!=j:
                f[i,j] = solver.IntVar(0.0, solver.infinity(), 'f[%i,%i]' % (i,j))

    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    
    [ solver.Add(  u[j] >= u[i] + x[i,j] - (n-2)*(1-x[i,j]) + (n-3)*x[j,i]   ) for i in otros for j in otros if j!=i ]
    
    [ solver.Add(  solver.Sum(f[i,j] for j in todos if i!=j) - solver.Sum(f[j,i] for j in todos if i!=j) == demanda[i]  )  for i in otros ]
    [ solver.Add(  max(0, demanda[i], -demanda[j]) * x[i,j] <= f[i,j]  )  for i in otros for j in otros if i!=j ]
    [ solver.Add(  f[i,j] <= (Q - max(0, -demanda[i], demanda[j])) * x[i,j]  )  for i in otros for j in otros if i!=j ]
    
    solver.Solve()

    print('Distancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('Ruta de mínima distancia:')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('De %d a %d con distancia=%d Km'  %(i,j,dist[i,j]))

TSP5()

**4.- Modelo del TSP con Ventanas Temporales**

In [0]:
!pip install pulp
from pulp import *
!pip install ortools
from ortools.linear_solver import pywraplp
import random

n = 50
todos = range(n)
otros = range(1,n)
random.seed(12345)
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
e = { i : random.randint(1,100) * i for i in otros }     #early time
l = { i : e[i]+random.randint(1,100) for i in otros }    #late time
t = { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }  #tiempo de ruta en cada arco
s = { i : random.randint(1,20) * i for i in otros }     #service time
EPS = 0.001

def TSP6():
    solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
    u = {}
    for i in otros:
        u[i] = solver.IntVar(0.0, solver.infinity(), 'u[%i]' % i) 
    f = {}
    for i in todos:
        for j in todos:
            if i!=j:
                f[i,j] = solver.IntVar(0.0, solver.infinity(), 'f[%i,%i]' % (i,j))
            
    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    
    [ solver.Add(  u[j] >= u[i] + x[i,j] - (n-2)*(1-x[i,j]) + (n-3)*x[j,i]  ) for i in otros for j in otros if j!=i ]
    
    '''[ solver.Add(  0 <= f[i,j]  )  for i in otros for i in otros if i!=j ]
    [ solver.Add(  f[i,j] <= T[i,j] * x[i,j]  )  for i in otros for i in otros if i!=j ]
    [ solver.Add(  solver.Sum(f[i,j] for j in todos) >= solver.Sum(f[k,i] + (s[k] + t[i,k]) * x[i,j] for k in todos)  )  for i in otros if i!=j and i!=k ]
    [ solver.Add(  e[i] <= solver.Sum(f[i,j] for j in todos)  )  for i in otros if i!=j ]
    [ solver.Add(  solver.Sum(f[i,j] for j in todos) <= l[i]  )  for i in otros if i!=j ]'''
    
    [ solver.Add(  e[i] <= u[i]  )  for i in otros]
    [ solver.Add(  u[i] <= l[i]  )  for i in otros]
    
    [ solver.Add(  u[j] <= u[i] + (s[i] + t[i,j]) * x[i,j] - n * (1 - x[i,j])  )  for i in otros for j in otros if i!=j ]
    
    
    solver.Solve()

    print('Distancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('Ruta de mínima distancia:')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('De %d a %d con distancia=%d Km'  %(i,j,dist[i,j]))

TSP6()